In [41]:
%matplotlib inline
import matplotlib.pyplot as plt
import image_slicer
from random import randint
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import numpy as np
from PIL import Image, ImageDraw
import datetime
import time
import os
import math
from image_slicer import Tile

def slice(filename, number_tiles, columns, rows):
    """
    Split an image into a specified number of tiles.
    Args:
       filename (str):  The filename of the image to split.
       number_tiles (int):  The number of tiles required.
       columns (int): The number of columns
       rows(int): The numebr of rows
    Returns:
        Tuple of :class:`Tile` instances.
    """
    im = Image.open(filename)

    im_w, im_h = im.size
    extras = (columns * rows) - number_tiles
    tile_w, tile_h = int(im_w // columns), int(im_h // rows)
    tiles = []
    number = 1
    for pos_y in range(0, im_h, tile_h): # -rows for rounding error.
        for pos_x in range(0, im_w, tile_w): # as above.
            area = (pos_x, pos_y, pos_x + tile_w, pos_y + tile_h)
            image = im.crop(area)
            position = (int(math.floor(pos_x / tile_w)) + 1,
                        int(math.floor(pos_y / tile_h)) + 1)
            coords = (pos_x, pos_y)
            tile = Tile(image, number, position, coords)
            tiles.append(tile)
            number += 1
    return tuple(tiles)

def join(tiles, columns, rows):
    """
    @param ``tiles`` - Tuple of ``Image`` instances.
    @return ``Image`` instance.
    """
    tile_size = tiles[0].image.size
    size = tile_size[0] * columns, tile_size[1] * rows
    im = Image.new('RGB', size, None)
    for tile in tiles:
        im.paste(tile.image, tile.coords)
    return im

st = datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d%H%M%S')
#img = "AerialFloyd_cropped.jpg"

if K.image_data_format() == 'channels_first':
    input_shape = (3, 50, 50)
else:
    input_shape = (50, 50, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.load_weights("simple_cnn_with_50X50_trial_1.h5")

In [42]:
im = Image.open("AerialFloyd_cropped.jpg")
x = im.size[0]
y = im.size[1]
rows = (x//50)
columns = (y//50)
numberOfTiles = rows * columns
tiles = slice("AerialFloyd_cropped.jpg",numberOfTiles,rows,columns)
x=[]

for i in range(0,numberOfTiles):
    tile = tiles[i]
    imgArray = np.array(tile.image)
    testImg = imgArray.reshape(input_shape)
    testImg=testImg/255.
    x.append(testImg)

x=np.array(x)
out = model.predict(x)

for i in range(0,numberOfTiles):
        tile = tiles[i]
        if out[i]<0.5:
            im = tile.image
            dr = ImageDraw.Draw(im)
            cor = (0,0, 50,50) # (x1,y1, x2,y2)
            line = (cor[0],cor[1],cor[0],cor[3])
            dr.line(line, fill="red", width=10)
            line = (cor[0],cor[1],cor[2],cor[1])
            dr.line(line, fill="red", width=10)
            line = (cor[0],cor[3],cor[2],cor[3])
            dr.line(line, fill="red", width=10)
            line = (cor[2],cor[1],cor[2],cor[3])
            dr.line(line, fill="red", width=10)

result = join(tiles,rows,columns)
result.save("simple_cnn_with_50X50_trial_2_conservative_.jpg")

NameError: name 'file' is not defined